<div style="text-align: right" align="right"><i>Peter Norvig, Sept 2022</i></div>

# Anigrams Puzzle

From 538's [**The Riddler** on Sept 16, 2022](https://fivethirtyeight.com/features/can-you-build-the-biggest-anigram/) comes this puzzle:
    
> In the game of [**Anigrams**](http://anigrams.us), you **unscramble** successively larger, nested collections of letters to create a valid **chain** of six English **words** between four and nine letters in length. For example, a **chain** of five words (sadly, less than the six needed for a valid game of Anigrams) can be constructed using the following sequence, with each **term** after the first including **one additional letter** than the previous **term**:
>
>- DEIR (which unscramble to make the words DIRE, IRED or RIDE)
>- DEIR**D** (DRIED or REDID)
>- DEIRD**L** (DIRLED, DREIDL or RIDDLE)
>- DEIRDL**R** (RIDDLER)
>- DEIRDLR**S** (RIDDLERS)
>
>
> What is the **longest chain** of such nested anagrams you can create, **starting** with four letters?
>
> *Extra credit:* **How many possible games** of Anigrams are there? That is, how many valid sets are there of four initial letters, and then five more **letters added** one at a time in an ordered sequence, that result in a sequence of six valid anagrams? (Note: Swapping the order of the first four letters does not result in a distinct game.)

I'll solve this puzzle with a Python program. First some generally useful imports:

In [1]:
from typing import Set, List
import functools 

## Basic Concepts

The puzzle description mentions a bunch of concepts: **word**, **chain**, **term**, **starting**, **unscramble**,  **longest chain**, and **one additional letter**. To solve the puzzle, I'll carefully define each concept, in prose and in Python. Once I have a collection of concepts, I can see how they fit together to solve the puzzle. First the data types:

- **Word**: an English language word, e.g. `'ride'`.
- **Chain**: a list of words, each one adding a letter, e.g. `['ride', 'redid', 'riddle']`. 
- **Term**: a scrambled word, e.g. `'deir'`. 

In [2]:
Word  = str        # A word in the English word list, e.g. 'ride'.
Chain = List[Word] # A list of words, each one adding a letter, e.g. ['ride', 'redid', 'riddle']. 
Term  = str        # A scrambled word, e.g. 'deir'

Then some functions and constants:

- **scramble**: I'll define `scramble(word)` to return one of the *n*! ways to scramble an *n*-letter word. I'll arbitrarily choose alphabetical order.
- **terms**: I'll define `all_terms` as a dict of `{Term: Word}` pairs, e.g.  `{'deir': 'ride', 'ddeir': 'redid', ...}`.
- **starting**: I'll define `starting_terms` to be the set of 4-letter terms from `all_terms`, e.g. `{'deir', 'acef', 'eirt', ...]`.
- **unscramble**: I'll define `unscramble(term)` to return the corresponding word. A term might unscramble to several words; I'll arbitrarily pick one.
- **longest chain**: I'll define`longest(chains)` to return the chain with the most words.
- **one additional letter**: I'll define `add_one_letter(term)` to return the set of valid terms that can be formed by adding one letter to `term`.

In [3]:
def scramble(word: Word) -> Term: 
    """A canonical scrambling of the letters in `word` (into alphabetical order)."""
    return ''.join(sorted(word))

all_terms = {scramble(w): w for w in open('enable1.txt').read().split()} # {Term: Word}

starting_terms = {term for term in all_terms if len(term) == 4} # 4-letter terms

unscramble = all_terms.get # e.g., unscramble('deir') = 'ride'

def longest(chains) -> Chain: return max(chains, key=len, default=[])

def add_one_letter(term: Term) -> Set[Term]:
    """The set of valid Terms (from `all_terms`) that can be formed by adding one letter to `term`."""
    terms = (scramble(term + L) for L in 'abcdefghijklmnopqrstuvwxyz')
    return {t for t in terms if t in all_terms} 

The following assertions serve as unit tests and also as examples of usage:

In [4]:
assert scramble('ride') == 'deir'
assert unscramble('deir') == 'ride'
assert scramble('dire') == scramble('ired') == scramble('ride')
assert unscramble('billowy') == scramble('billowy') == 'billowy'
assert longest(([1, 2, 3], [1, 2], [1, 2, 3, 4], [3, 2, 1])) == [1, 2, 3, 4]
assert longest([]) == [] # The longest of no chains is the empty chain
assert add_one_letter('abck') == {'aabck', 'abckl', 'abcks'}
assert add_one_letter(scramble('back')) == {scramble('aback'), scramble('black'), scramble('backs')}
assert add_one_letter(scramble('riddler')) == {scramble('riddlers')}
assert add_one_letter(scramble('riddlers')) == set()
assert len(all_terms) == 156473
assert len(starting_terms) == 2674

## Solution

We are now ready to compute "the longest chain ... starting with four letters." First we define `longest_starting_from(term)` to compute the longest chain starting from a term, then we apply that to every `starting_term`, and take the `longest`. To compute `longest_starting_from(term)`, recursively apply `longest_starting_from` to every one-letter addition to `term`, and then return the longest of those chains, with the unscrambling of `term` prepended. 

In [5]:
cache = functools.lru_cache(None) # Used as a function decorator to avoid re-computation

@cache
def longest_starting_from(term: Term) -> Chain:
    """The longest chain starting with `term`. Equal to the unscrambling of `term`,
    followed by the longest chain starting from a one-letter addition to `term` (if there are any)."""
    chains = map(longest_starting_from, add_one_letter(term))
    return [unscramble(term), *longest(chains)]

%time longest(map(longest_starting_from, starting_terms)) # A solution

CPU times: user 1.35 s, sys: 9.73 ms, total: 1.36 s
Wall time: 1.36 s


['nine',
 'inane',
 'eonian',
 'enation',
 'nominate',
 'nominates',
 'antinomies',
 'nitrosamine',
 'terminations',
 'antimodernist',
 'determinations',
 'underestimation',
 'underestimations']

We see that the longest chain gets up to a 16-letter word. There are other chains that are equally long, and it turns out that all of them end with either `indeterminations` or `underestimations`. 

*Note*: Typically a recursive function has the form "if terminal_condition return something else recurse." The function `longest_starting_from` does not need an explicit check for a terminal condition, because when `add_one_letter(term)` returns the empty set, the function `longest` deals with it (via the `default=[]` keyword).

## Extra Credit

How many possible games of Anigrams are there? A game is a chain that starts with a 4-letter word and ends with a 9-letter word. Swapping the order of letters in any word "does not result in a distinct game." (I don't have to worry about that because my `unscramble(term)` always picks the same word, given a term.)

The number of games starting from a given term is exactly 1 if the term has 9 letters, and is otherwise the sum of the number of games starting from each one-letter addition to term. To find the total number of possible games, just sum up the `games_starting_from` for each 4-letter term:

In [6]:
@cache
def games_starting_from(term: Term, goal_length=9) -> int:
    """How many games start with `term` and end with a word of `goal_length` letters?"""
    if len(term) == goal_length:
        return 1
    else:
        return sum(games_starting_from(t, goal_length) for t in add_one_letter(term))

In [7]:
%time sum(map(games_starting_from, starting_terms))

CPU times: user 816 ms, sys: 4.69 ms, total: 821 ms
Wall time: 820 ms


4510515

There are 4,510,515 total possible games. 

## More Tests, Examples, and Notes



In [11]:
assert games_starting_from('beep') == 1
assert longest_starting_from('beep') == ['beep', 'plebe', 'beleap', 'beleapt', 'bedplate', 'bedplates']
assert max([games_starting_from(term), term] for term in starting_terms) == [64422, 'aers']
for word in 'alerts alters artels estral laster ratels salter slater staler stelar talers'.split():
    assert scramble(word) == 'aelrst' 
for (term, word) in all_terms.items():
    assert scramble(word) == term
    assert unscramble(term) == word
    assert scramble(unscramble(term)) == term
    assert unscramble(scramble(word)) == word

The words `'a'` and `'I'` are not in `enable1.txt`, because one-letter words are not valid Scrabble™️ words. We could add them, and then find the longest chain of all:

In [9]:
all_terms.update({'a': 'a', 'i': 'i'})
longest(map(longest_starting_from, all_terms))

['a',
 'as',
 'sea',
 'teas',
 'teams',
 'stamen',
 'tameins',
 'mannites',
 'nominates',
 'antinomies',
 'nitrosamine',
 'terminations',
 'antimodernist',
 'determinations',
 'underestimation',
 'underestimations']

*Note*: The puzzle description uses the phrase "collection of letters" and the word "anagram" as synonyms for "term". I chose to use "term" throughout because "collection of letters" is too long, and "anagram" (although more descriptive than the bland "term") is not quite right: "ride" and "dire" are indeed anagrams of each other, but "deir" is not an anagram, because it is not an English word. Rather, "deir" is a *scrambled collection of letters that correspond to a word*. Also, I like the fact that "term" and "word" have the same number of letters.

*Note*: This whole notebook only takes about 3 seconds to run. The use of the `@cache` decorator means that the complexity of the whole algorithm is *O*(*T*), where *T* is the number of terms. Without the `@cache`, many chains would have to be re-computed multiple times. For example, starting at `'ride'` you can add one letter to get to either `'riced'` or `'rimed'`. From either of those you can add one letter to get to `'dermic'`, and from there there's a whole subtree of possibilities to explore. With `@cache`, that subtree is only computed once. Without `@cache` it is computed twice, and lots of other subtrees are computed multiple times, making the overall run time about ten-fold slower.

*Note*: I could have represented a chain as a list of scrambled terms, rather than a list of words, but I figure if you want the solution, you'd rather see words, not scrambles. I also could have represented a chain as an initial set of four letters plus the letters that get added, in order. For example, `'deir' + 'd' + 'l' + 'r' + 's' == 'deirdlrs'`. That would have been a more compact representation than a list, but messier to deal with. For example, the word `'dermic'` would be reached by two paths, `'deircm'` and `'deirmc'`, and we would lose the advantage of `@cache` unless we took some convoluted steps to translate back and forth among the chain format, the scrambled word format, and the unscrambled word format.

*Note*: Because the order of iteration over sets is not fixed in Python, different runs of this notebook can produce different results.

